In [1]:
import tensorflow
import numpy

print(tensorflow.__version__)
print(numpy.__version__)

2023-10-25 20:35:46.582918: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-10-25 20:35:46.673354: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-10-25 20:35:46.673368: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-10-25 20:35:47.194297: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

2.11.0
1.22.0


In [1]:
import metaworld
# import environments.metaworld_envs
import gym
from environments.parallel_envs import make_vec_envs, make_env


In [2]:
ml10 = metaworld.ML10()

In [3]:
ml10_env = gym.make('ML10-v2')

In [4]:
# env = gym.make('ML10-v2')
varibad_env = make_env('ML10-v2', 1, 1, 2, tasks = ml10.train_classes, add_done_info =True)

In [5]:
vbad_ml10 = varibad_env()

In [7]:
vbad_ml10.reset_mdp()


array([ 0.00615235,  0.6001898 ,  0.19430117,  1.        ,  0.26273011,
        0.73441412,  0.15003595,  0.6907585 , -0.15062097,  0.69087866,
        0.15117109,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.00615235,  0.6001898 ,
        0.19430117,  1.        ,  0.26273011,  0.73441412,  0.15003595,
        0.6907585 , -0.15062097,  0.69087866,  0.15117109,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [9]:
vbad_ml10.observation_space


Box(40,)